##### installing packages

In [5]:
!pip install transformers torch evaluate tqdm dataset

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.37
    Uninstalling SQLAlchemy-2.0.37:
      Successfully uninstalled SQLAlchemy-2.0.37
  Attempting uninstall: fsspec
    Found exis

#### Loading packages

In [6]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import re
import pandas as pd
import string
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import torch, codecs, random
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from evaluate import load as load_metric
from google.colab import output
import matplotlib.pyplot as plt
from typing import List, Dict, Any
from transformers import PreTrainedTokenizer
from torch.optim import Optimizer

output.enable_custom_widget_manager()

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


## Data Preprocessing

In [8]:
df = pd.read_csv("/content/drive/MyDrive/Tweets_5K.csv")

df.head()

,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative


In [9]:
df['sentiment'].value_counts()

,count
sentiment,
neutral,2023
positive,1592
negative,1385


In [11]:
df['sentiment'] = df['sentiment'].replace({'neutral': 0,
                                           'positive': 1,
                                           'negative': -1})

raw_tweets = df['text'].to_list()
labels = df['sentiment'].to_list()


# X = dataset['text'].to_list()
# y = dataset['sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2}).to_list()

In [17]:
#Featurize - bag of words

tweet_dicts = [dict(Counter(tweet)) for tweet in basic_preproc_tweets]
vectorizer = DictVectorizer(sparse=False)
X = vectorizer.fit_transform(tweet_dicts)
vocabulary = vectorizer.get_feature_names_out()

print("Vocabulary:\n", vocabulary)

Vocabulary:
 ['!' '!!' '!!!' ... 'ï¿½4.80' 'ï¿½anisalovesu' 'ï¿½you']


In [18]:
df_bow = pd.DataFrame(X, columns=vocabulary)
df_bow.head(5)

,!,!!,!!!,!!!!,!!!!!,!!!!!!!!,!!my,!+,!1!!,#,...,zombies?',{this,{{{HUGS}}},~,ï¿½,ï¿½1.50,ï¿½2,ï¿½4.80,ï¿½anisalovesu,ï¿½you
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Train Test Split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,labels,test_size = 0.2, random_state=42)

X_train.shape

(4000, 15121)